In [1]:
import pandas as pd
import numpy as np

In [2]:
import os 

os.chdir('../')

In [3]:
movies = pd.read_csv('data/ml-latest-small/movies.csv')

ratings = pd.read_csv('data/ml-latest-small/ratings.csv')

In [9]:
movie_mapper = np.sort(np.array(movies.movieId.unique()))

def get_movie_hash(id):
    return np.where(movie_mapper == id)[0][0]

def get_movie_id(id):
    return movie_mapper[id]

In [11]:
from sklearn.neighbors import NearestNeighbors
import random

def collaborative_filtering(R, id, k=10):
    # Get k nearest neighbors
    kNN = NearestNeighbors(n_neighbors=20+1, algorithm="brute", metric='cosine')
    kNN.fit(R)
    # Get indices of neighbors (excluding the user themselves)
    distances, indices = kNN.kneighbors(R[id].reshape(1,-1), return_distance=True)
    neighbor_indices = indices[0][1:]
    neighbor_distances = distances[0][1:]
    
    # Convert distances to weights (similarity scores)
    # Using inverse distance weighting
    similarities = 1 / (neighbor_distances + 1e-8)  # add small number to avoid division by zero
    # Normalize weights
    similarities = similarities / np.sum(similarities)
    
    # Get user's current ratings
    user_ratings = R[id]
    unrated_movies = user_ratings == 0
    
    # Calculate predicted ratings
    predicted_ratings = np.zeros_like(user_ratings)
    for idx, neighbor_idx in enumerate(neighbor_indices):
        neighbor_ratings = R[neighbor_idx]
        # Only consider movies the neighbor has rated
        rated_by_neighbor = neighbor_ratings != 0
        # Weight the neighbor's ratings by their similarity
        predicted_ratings += similarities[idx] * neighbor_ratings
    
    # Only keep predictions for movies the user hasn't rated
    predicted_ratings[~unrated_movies] = 0
    
    # Get top 10 movie recommendations
    top_movies_indices = np.argsort(predicted_ratings)[::-1][:k]
    
    # Create recommendations list with (movie_id, predicted_rating)
    recommendations = [idx for idx in top_movies_indices]
    
    return recommendations

In [15]:
class Environment:
    def __init__(self, ratings_df, movies_df):
        self.ratings = ratings_df.copy()  # Create a copy of the ratings DataFrame
        self.movies = movies_df.copy()
        
class ReflexAgent:
    def __init__(self, id, env, k=50):
        self.id = id
        self.env = env  # Store reference to environment
        mh = list(self.env.ratings[self.env.ratings['userId'] == self.id]
                 .sort_values('timestamp').head(k).movieId)
        self.movie_history = list(map(get_movie_hash, mh))
    
    def get_recs(self, R):
        self.recs = collaborative_filtering(R, self.id)
        return self.recs
    
    def action(self):
        self.chosen = random.sample(self.recs, 1)[0]
        self.movie_history.append(self.chosen)

        # Create a new row as a DataFrame
        new_rating = pd.DataFrame({
            'userId': [self.id],
            'movieId': [get_movie_id(self.chosen)],
            'rating': [random.randint(0, 10)/2],
            'timestamp': [82983923]
        })
    
        # Update the ratings DataFrame in the environment
        self.env.ratings = pd.concat([self.env.ratings, new_rating], ignore_index=True)
        return self.chosen

def create_R(agents, env):
    m = len(env.movies.movieId)
    R = []
    for agent in agents:
        mh = agent.movie_history
        mh_idx = list(map(get_movie_id, mh))
        user_ratings = env.ratings[(env.ratings['userId'] == agent.id) & 
                                 (env.ratings['movieId'].isin(mh_idx))]
        
        # Modified row creation with error handling
        row = []
        for i in range(m):
            if i in mh:
                # Get ratings for this movie
                movie_ratings = user_ratings[user_ratings['movieId'] == get_movie_id(i)].rating
                # Check if we have any ratings
                if len(movie_ratings) > 0:
                    row.append(float(movie_ratings.iloc[0]))
                else:
                    row.append(0)  # No rating found
            else:
                row.append(0)  # Movie not in history
                
        # Convert to numpy array for easier manipulation
        row_array = np.array(row)
        
        # Calculate mean of only non-zero elements
        non_zero_mask = row_array != 0
        if non_zero_mask.any():  # check if there are any non-zero elements
            mean = np.mean(row_array[non_zero_mask])
            # Subtract mean only from non-zero elements
            row_array[non_zero_mask] = row_array[non_zero_mask] - mean
        
        R.append(row_array.tolist())

    return np.array(R)



In [16]:
# Create the environment
env = Environment(ratings, movies)

# Initialize agents with environment
users = np.array(pd.Series(ratings.userId.unique()).sample(n=200, random_state=42))
agents = [ReflexAgent(list(users).index(i), env) for i in users]

# Run simulation
for t in range(5):
    print(f"Creating R matrix for timestep {t}")
    R = create_R(agents, env)
    print(f"R matrix shape: {R.shape}")
    for i, agent in enumerate(agents):
        print(f"Processing agent {i} with {len(agent.movie_history)} movies in history")
        rec = agent.get_recs(R)
        agent.action()
    print(f"Completed timestep: {t}")

Creating R matrix for timestep 0
R matrix shape: (200, 9742)
Processing agent 0 with 0 movies in history
Processing agent 1 with 50 movies in history
Processing agent 2 with 29 movies in history
Processing agent 3 with 39 movies in history
Processing agent 4 with 50 movies in history
Processing agent 5 with 44 movies in history
Processing agent 6 with 50 movies in history
Processing agent 7 with 50 movies in history
Processing agent 8 with 47 movies in history
Processing agent 9 with 46 movies in history
Processing agent 10 with 50 movies in history
Processing agent 11 with 50 movies in history
Processing agent 12 with 32 movies in history
Processing agent 13 with 31 movies in history
Processing agent 14 with 48 movies in history
Processing agent 15 with 50 movies in history
Processing agent 16 with 50 movies in history
Processing agent 17 with 50 movies in history
Processing agent 18 with 50 movies in history
Processing agent 19 with 50 movies in history
Processing agent 20 with 50 mo